In [ ]:
!pip install transformers datasets tf-models-official

In [ ]:
from psutil import virtual_memory
from google.colab import files
import pandas as pd
import numpy as np
import io
import nltk
import re
import string
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import DataCollatorWithPadding
from transformers import TFAutoModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import Dataset
import tensorflow as tf
import tensorflow_models as tfm
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
uploaded = files.upload()

Saving multi_train_labels.csv to multi_train_labels.csv


In [ ]:
SEED = 2

In [ ]:
task2_df = pd.read_csv(io.BytesIO(uploaded['multi_train_labels.csv']))

In [ ]:
label_cols = ['G','L','B','T','O']
task2_df['labels'] = list(task2_df[label_cols].values)

In [ ]:
task2_df

,tuit,G,L,B,T,O,labels
0,"—Bueno, pues ya pasó la euforia papá, ahora sí...",1,0,0,0,0,"[1, 0, 0, 0, 0]"
1,"Esos de BTS se me hacen seres asexuales, eunuc...",0,0,0,0,1,"[0, 0, 0, 0, 1]"
2,La partida de skywars que me ha hecho enojar m...,1,0,0,0,0,"[1, 0, 0, 0, 0]"
3,"Se te acabo tu juego maldito joto asqueroso, e...",1,0,0,0,0,"[1, 0, 0, 0, 0]"
4,@Lunaazulada_ Será a los jotitos.??ellos prefi...,1,0,0,0,0,"[1, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
857,@alf_avila_ @jrisco @penileyramirez De mi part...,1,0,0,0,0,"[1, 0, 0, 0, 0]"
858,@CxavezE @altazor08 Se acalambra el putito 🤷🏽‍...,1,0,0,0,0,"[1, 0, 0, 0, 0]"
859,"Psst, solo les quiero spoilerear la platica qu...",0,0,0,0,1,"[0, 0, 0, 0, 1]"
860,@navirri8 @DemonCerda @MARIOHDZRUIZ ps no somo...,1,0,0,0,1,"[1, 0, 0, 0, 1]"


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(list(task2_df['tuit']), task2_df['labels'],
                                                                    test_size=.15, random_state=SEED, shuffle=True)

In [ ]:
print(len(train_texts))
print(len(val_texts))

732
130


In [ ]:
checkpoint = "dccuchile/bert-base-spanish-wwm-cased" 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
batch_size = 4
num_epochs = 4

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler)
loss=BinaryCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5, problem_type="multi_label_classification", 
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.2,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"],
)

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/4
183/183 [==============================] - 107s 274ms/step - loss: 0.2819 - accuracy: 0.8115 - val_loss: 0.2043 - val_accuracy: 0.8462
Epoch 2/4
183/183 [==============================] - 33s 179ms/step - loss: 0.1764 - accuracy: 0.8689 - val_loss: 0.1475 - val_accuracy: 0.9154
Epoch 3/4
183/183 [==============================] - 32s 177ms/step - loss: 0.1291 - accuracy: 0.9235 - val_loss: 0.1336 - val_accuracy: 0.9231
Epoch 4/4
183/183 [==============================] - 30s 165ms/step - loss: 0.0997 - accuracy: 0.9454 - val_loss: 0.1318 - val_accuracy: 0.9308


In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = [(r > 0).astype(np.int) for r in preds]

In [ ]:
print("Beto Accuracy Score:", round(accuracy_score(class_preds, list(val_encodings['labels'])), 2))

Beto Accuracy Score: 0.86


In [ ]:
print(classification_report(class_preds, list(val_encodings['labels'])))

              precision    recall  f1-score   support

           0       0.96      0.97      0.97       109
           1       0.78      0.58      0.67        12
           2       0.00      0.00      0.00         0
           3       0.69      0.90      0.78        10
           4       0.12      0.50      0.20         2

   micro avg       0.87      0.92      0.89       133
   macro avg       0.51      0.59      0.52       133
weighted avg       0.91      0.92      0.92       133
 samples avg       0.91      0.93      0.92       133



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
